In [ ]:
#Import des modules nécessaires à savoir requests pour créer,utiliser et excécuter nos requêtes, selenium comme recommandé, time pour permettre le chargement des pages du site et sys pour pouvoir quitter le systeme de la boucle dès qu'indiquer. 
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import sys

In [ ]:
#chemin complet vers chromedriver.exe
path_to_chromedriver = './chromedriver-win32/chromedriver.exe'

# Création d'une instance de Service en spécifiant le chemin du chromedriver
chrome_service = Service(path_to_chromedriver)

# Création du navigateur en utilisant l'instance de Service
driver = webdriver.Chrome(service=chrome_service)

# Naviguation vers l'URL souhaitée
driver.get('https://anatta.ca/')

In [ ]:
# Recherche du nombre de rubriques de vêtements présentes sur la barre de navigation du site afin de connaître le nombre de pages à scraper:
time.sleep(2)
rubriques = driver.find_elements(By.XPATH, "//div[@class='t4s-mb-tab__content is--active']//div[@class='shopify-section t4s-sp-section-mb-nav']//ul[@class='t4s-mb__menu']//li[@class='t4s-menu-item t4s-item-level-0']")

nb_pages = len(rubriques)  # Nombre de pages stocké dans la variable nb_pages
print("Il y a actuellement " + str(nb_pages) + " catégories sur ce site. \nLes voici :")  # Affichage du nombre de pages présentes pour la catégorie sélectionnée

# Nous avons la liste genres qui contiendra les différents genres de rubriques
genres = []
for collections in driver.find_elements(By.XPATH, "//div[@class='t4s-mb-tab__content is--active']//div[@class='shopify-section t4s-sp-section-mb-nav']//ul[@class='t4s-mb__menu']//li[@class='t4s-menu-item t4s-item-level-0']"):
    genre = collections.find_element(By.TAG_NAME, 'a').get_attribute('href')
    genres.append(genre)

genres.pop(0)  # Supprimer le premier élément
genres.pop(-2)
genres.pop(-1)
print(genres)  # Afficher le lien directement

In [ ]:
produits = [] #Création du tableau où seront stocker les données.
for page in genres: #Pour le nombre de pages présentes dans la catégorie
    i = 1 #Initialisation d'une variable i qui sera incrémenter à chaque tour de la boucle afin d'accéder aux diverses produits présents.
    datas = 0
    driver.get(page)
    
    for data in range(500):  #Limitation des produits à afficher
        try: #Essayer de...
            #Récupérer les diverses données             
            produit = driver.find_element("xpath", f'/html/body/div[2]/main/section[2]/div/div/div/div/div[1]/div{[i]}/div/div[2]/div/h3/a').text            
            prix = driver.find_element("xpath", f'/html/body/div[2]/main/section[2]/div/div/div/div/div[1]/div{[i]}/div/div[2]/div/div[1]').text 
            
            #time.sleep(3) #Simplement pour l'exécution, permettre un tant de chargement des images afin de les récupérer
            image_element = driver.find_element("xpath", f'/html/body/div[2]/main/section[2]/div/div/div/div/div[1]/div{[i]}/div/div[1]/div[1]/img[1]')
            image_src = image_element.get_attribute('data-srcset')
            
            #Afficher les diverses données
            print(produit)
            print(prix)
            print(image_src)
            
            #Stocker les diverses données dans la table
            produits.append([produit, prix, image_src])

            #Initialisation du scroll vers le bas à l'aide de la bibliothèque ActionsChains qui simule la touche d'appui vers le bas
            actions = ActionChains(driver)
            for scroll in range(5):  #Initialiser le nombre d'appui
                actions.send_keys(Keys.ARROW_DOWN)  # Utiliser la touche fléchée vers le bas
                actions.perform()
            
            try: #Essayer de...
                #Fermer la fenetre de publicité si elle active pour pouvoir accéder aux produits
                close_ad_button = driver.find_element(By.CLASS_NAME, 't4smfp-icon-close')
                close_ad_button.click()
            except Exception: #Si ça ne marches pas
                pass  # Ignorer et continuer 

            
            try: #Essayer de...
                #Charger plus d'éléments en cliquant sur le bouton "Voir plus"
                load= driver.find_element("xpath", f'/html/body/div[2]/main/section[2]/div/div/div/div/div[2]/div/a')
                load.click()
            except Exception: #Si ça ne fonctionnes pas
                pass #Ignorer et continuer

        except Exception: #Si l'essaye échoue..
            break  #Sortir de la boucle et passer à autre chose
        i += 1 #On incrémente pour passer à l'indice suivant
        datas += 1 #On incrémente pour dire qu'une donnée supplémentaire vient d'etre collecter
    print(str(datas) + " données viennent d'être collectées.")
    time.sleep(3)

print("Actuellement, " + str(len(produits)) + " données ont pu être récupérées.")  #Afficher le nombre d'élément présente dans la liste

In [ ]:
# Utilisation de join pour séparer par des points-virgules
resultat = ";".join([";".join(map(str, element)) for element in produits]) #Séparer les données récupérées par des ";" afin de pouvoir récupérer les données, les stocker dans une table CSV et les importer dans une base de données
print(resultat) # Affichage du résultat

driver.quit() # Fermer le navigateur 